<a href="https://colab.research.google.com/github/luismartins26/segcomp/blob/main/Trabalho_Pr%C3%A1tico_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Prático 01

CIC0201 - Segurança Computacional – 2025/1

Disciplina: Segurança Computacional

Professora: Lorena Borges

Estudante: Luís Martins (242037814)

# Introdução

O trabalho está disponível em https://colab.research.google.com/drive/198A1oyLfz-MXqa149wFRMlNZrSqWfhYv?usp=sharing e

Este trabalho contou com o auxílio de ferramentas de Inteligência Artificial (OPENAI, 2023; GOOGLE, 2024) para a geração de funções auxiliares e revisão do texto escrito. Todo o conteúdo gerado foi cuidadosamente revisado, ajustado e validado, garantindo a sua adequação, coerência e conformidade com os objetivos do trabalho.

In [ ]:
from abc import ABC, abstractmethod
import math
from collections import OrderedDict
import random

In [ ]:
# prompt: configure logger and log a sample message

import logging

# Configure the logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to write log messages to a file (optional)
file_handler = logging.FileHandler('sdes.log')  # Creates or overwrites the file
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [ ]:
class Modulo(ABC):
    @abstractmethod
    def executar(self, T : str) -> str:
        pass

In [ ]:
class Permutador(Modulo):
    def __init__(self, chave):
        self.key = chave

    def executar(self, T : str) -> str:
        # Cria a lista C preenchida com espaços para armazenar o texto cifrado
        C = [' ']*math.ceil(len(T)/len(self.key))*len(self.key)
        # Preenche a mensagem original com espaços extras
        T = T+' '*(math.ceil(len(T)/len(self.key))*len(self.key)-len(T))
        # Para cada bloco de tamanho igual ao da chave
        for j in range(math.ceil(len(T)/len(self.key))):
            # Aplica a permutação no bloco
            for i, k in enumerate(self.key):
                C[j*len(self.key)+i] = T[j*len(self.key)+k]
        # Junta a lista de caracteres em uma string e retorna o texto cifrado
        return "".join(C)

In [ ]:
class Selecionador(Modulo):
    def __init__(self, chave):
        self.key = chave

    def executar(self, T : str) -> str:
        T_ = []

        for i in self.key:
            T_.append(T[i])

        return "".join(T_)

In [ ]:
def binario_para_numerico(T : str) -> int:
    return int(T, 2)

def numerico_para_binario(n: int) -> str:
    return bin(n)[2:]

In [ ]:
def xor(s1: str, s2: str) -> str:
    return numerico_para_binario(binario_para_numerico(s1) ^ binario_para_numerico(s2)).zfill(len(s1))

In [ ]:
class CaixaSubstituicao(Modulo):
    def __init__(self, chave):
        self.key = chave

    def executar(self, T : str) -> str:
        p1 = binario_para_numerico(T[0]+T[3])
        p2 = binario_para_numerico(T[1]+T[2])
        r = self.key[p1][p2]
        return numerico_para_binario(r).zfill(2)

In [ ]:
class RodadaFeistel(Modulo):
    def set_EP_key(self, key):
        self.EP_key = key

    def set_S1_key(self, key):
        self.S1_key = key

    def set_S2_key(self, key):
        self.S2_key = key

    def set_P4_key(self, key):
        self.P4_key = key

    def set_subkey(self, subkey):
        self.subkey = subkey

    def executar(self, T : str) -> str:
        logger.info(f'\tRodada Feistel de {T} com subchave {self.subkey}')
        self.fabricar_modulos()
        L = T[:4]
        R = T[4:]
        T = self.modulos['EP'].executar(R)
        logger.info(f'\tApós a realização da operação EP, a sequência resultante é {T}')
        T = xor(T, self.subkey)
        logger.info(f'\tApós a realização da operação XOR, a sequência resultante é {T}')
        T1 = self.modulos['S1'].executar(T[:4])
        logger.info(f'\tApós a realização da operação S1, a sequência resultante é {T1}')
        T2 = self.modulos['S2'].executar(T[4:])
        logger.info(f'\tApós a realização da operação S2, a sequência resultante é {T2}')
        T = T1+T2
        T = self.modulos['P4'].executar(T)
        logger.info(f'\tApós a realização da operação P4, a sequência resultante é {T}')
        T = xor(L, T)
        logger.info(f'\tApós a realização da operação XOR, a sequência resultante é {R}')
        return T+R

    def fabricar_modulos(self):
        self.modulos = OrderedDict()
        self.modulos['EP'] = Permutador(self.EP_key)
        self.modulos['S1'] = CaixaSubstituicao(self.S1_key)
        self.modulos['S2'] = CaixaSubstituicao(self.S2_key)
        self.modulos['P4'] = Permutador(self.P4_key)


In [ ]:
class FabricaModulo():
    def fabricar_permutador(self, chave) -> Permutador:
        p = Permutador(chave)
        return p

    def fabricar_selecionador(self, chave) -> Selecionador:
        p = Selecionador(chave)
        return p

    def fabricar_caixa_substituicao(self, chave) -> CaixaSubstituicao:
        p = CaixaSubstituicao(chave)
        return p

    def fabricar_rodada_feistel(self) -> RodadaFeistel:
        p = RodadaFeistel()
        p.set_EP_key([3, 0, 1, 2, 1, 2, 3, 0])
        p.set_S1_key([[1, 0, 3, 2], [3, 2, 1, 0], [0, 2, 1, 3], [3, 1, 3, 2]])
        p.set_S2_key([[0, 1, 2, 3], [2, 0, 1, 3], [3, 0, 1, 0], [2, 1, 0, 3]])
        p.set_P4_key([1, 3, 2, 0])

        return p

In [ ]:
class Cifra(ABC):
    @abstractmethod
    def Gen(self) -> any:
        pass

    def set_key(self, key):
        self.key = key

    @abstractmethod
    def Enc(self, M: str) -> str:
        pass

    @abstractmethod
    def Dec(self, C: str) -> str:
        pass

In [ ]:
class SDES(Cifra):
    def criar_gerador_subchaves(self):
        self.gerador_subchaves = OrderedDict()
        P10 = self.fabrica.fabricar_permutador([2, 4, 1, 6, 3, 9, 0, 8, 7, 5])
        self.gerador_subchaves['P10'] = P10
        S1 = self.fabrica.fabricar_permutador([1, 2, 3, 4, 0])
        self.gerador_subchaves['S1'] = S1
        P8 = self.fabrica.fabricar_selecionador([5, 2, 6, 3, 7, 4, 9, 8])
        self.gerador_subchaves['P8'] = P8
        S2 = self.fabrica.fabricar_permutador([2, 3, 4, 0, 1])
        self.gerador_subchaves['S2'] = S2

    def __init__(self):
        self.fabrica = FabricaModulo()
        self.criar_gerador_subchaves()
        self.criar_encriptador()

    def set_key(self, key):
        self.key = key
        self.gerar_subchaves(key)
        self.encriptador['F1'].set_subkey(self.subchaves['K1'])
        self.encriptador['F2'].set_subkey(self.subchaves['K2'])

    def criar_encriptador(self):
        self.encriptador = OrderedDict()
        IP = self.fabrica.fabricar_permutador([1, 5, 2, 0, 3, 7, 4, 6])
        self.encriptador['IP'] = IP
        F1 = self.fabrica.fabricar_rodada_feistel()
        self.encriptador['F1'] = F1
        SW = self.fabrica.fabricar_permutador([4, 5, 6, 7, 0, 1, 2, 3])
        self.encriptador['SW'] = SW
        F2 = self.fabrica.fabricar_rodada_feistel()
        self.encriptador['F2'] = F2
        IP_1 = self.fabrica.fabricar_permutador([3, 0, 2, 4, 6, 1, 7, 5])
        self.encriptador['IP_1'] = IP_1

    def Gen(self) -> str:
        k = random.getrandbits(10)
        k = numerico_para_binario(k)
        k = k.zfill(10)
        return k

    def gerar_subchaves(self, K : str) -> OrderedDict:
        logger.info(f'Gerando subchaves a partir da chave {K}')
        self.subchaves = OrderedDict()
        T = self.gerador_subchaves['P10'].executar(K)
        logger.info(f'Após a realização da operação P10, a sequência resultante é {T}')
        T = self.gerador_subchaves['S1'].executar(T)
        logger.info(f'Após a realização da operação S1, a sequência resultante é {T}')
        self.subchaves['K1'] = self.gerador_subchaves['P8'].executar(T)
        logger.info(f'Após a realização da operação P8, a subchave K1 resultante é {self.subchaves["K1"]}')
        T = self.gerador_subchaves['S2'].executar(T)
        logger.info(f'Após a realização da operação S2, a sequência resultante é {T}')
        self.subchaves['K2'] = self.gerador_subchaves['P8'].executar(T)
        logger.info(f'Após a realização da operação P8, a subchave K2 resultante é {self.subchaves["K2"]}')

    def Enc(self, M: str) -> str:
        logger.info(f'Encriptando a mensagem {M}')
        T = self.encriptador['IP'].executar(M)
        logger.info(f'Após a realização da operação IP, a sequência resultante é {T}')
        T = self.encriptador['F1'].executar(T)
        logger.info(f'Após a realização da operação F1, a sequência resultante é {T}')
        T = self.encriptador['SW'].executar(T)
        logger.info(f'Após a realização da operação SW, a sequência resultante é {T}')
        T = self.encriptador['F2'].executar(T)
        logger.info(f'Após a realização da operação F2, a sequência resultante é {T}')
        C = self.encriptador['IP_1'].executar(T)
        logger.info(f'Após a realização da operação IP_1, a sequência resultante é {C}')
        return C

    def Dec(self, C: str) -> str:
        logger.info(f'Decriptando a mensagem {C}')
        T = self.encriptador['IP'].executar(C)
        logger.info(f'Após a realização da operação IP, a sequência resultante é {T}')
        T = self.encriptador['F2'].executar(T)
        logger.info(f'Após a realização da operação F2, a sequência resultante é {T}')
        T = self.encriptador['SW'].executar(T)
        logger.info(f'Após a realização da operação SW, a sequência resultante é {T}')
        T = self.encriptador['F1'].executar(T)
        logger.info(f'Após a realização da operação F1, a sequência resultante é {T}')
        M = self.encriptador['IP_1'].executar(T)
        logger.info(f'Após a realização da operação IP_1, a sequência resultante é {M}')
        return M


In [ ]:
sdes = SDES()
sdes.set_key('1010000010')
C = sdes.Enc('11010111')
M = sdes.Dec(C)
assert M == '11010111'

INFO:root:Gerando subchaves a partir da chave 1010000010
INFO:root:Após a realização da operação P10, a sequência resultante é 1000001100
INFO:root:Após a realização da operação S1, a sequência resultante é 0000111000
INFO:root:Após a realização da operação P8, a subchave K1 resultante é 10100100
INFO:root:Após a realização da operação S2, a sequência resultante é 0010000011
INFO:root:Após a realização da operação P8, a subchave K2 resultante é 01000011
INFO:root:Encriptando a mensagem 11010111
INFO:root:Após a realização da operação IP, a sequência resultante é 11011101
INFO:root:	Rodada Feistel de 11011101 com subchave 10100100
INFO:root:	Após a realização da operação EP, a sequência resultante é 11101011
INFO:root:	Após a realização da operação XOR, a sequência resultante é 01001111
INFO:root:	Após a realização da operação S1, a sequência resultante é 11
INFO:root:	Após a realização da operação S2, a sequência resultante é 11
INFO:root:	Após a realização da operação P4, a sequência 

In [ ]:
class ModoOperacao(ABC):
    @abstractmethod
    def encriptar(self, M: str) -> str:
        pass

    @abstractmethod
    def decriptar(self, C: str) -> str:
        pass

    def __init__(self, cifra: Cifra):
        self.cifra = cifra

In [ ]:
class ECB(ModoOperacao):
    def set_tamanho_bloco(self, tamanho_bloco):
        self.tamanho_bloco = tamanho_bloco

    def encriptar(self, M: str) -> str:
        C = ''
        for i in range(0, len(M), self.tamanho_bloco):
            C += self.cifra.Enc(M[i:i+self.tamanho_bloco])
        return C

    def decriptar(self, C: str) -> str:
        M = ''
        for i in range(0, len(C), self.tamanho_bloco):
            M += self.cifra.Dec(C[i:i+self.tamanho_bloco])
        return M

In [ ]:
sdes = SDES()
sdes.set_key('1010000010')
ecb = ECB(sdes)
ecb.set_tamanho_bloco(8)
M = '11010111011011001011101011110000'
C = ecb.encriptar(M)
M_ = ecb.decriptar(C)
assert M == M_

INFO:root:Gerando subchaves a partir da chave 1010000010
INFO:root:Após a realização da operação P10, a sequência resultante é 1000001100
INFO:root:Após a realização da operação S1, a sequência resultante é 0000111000
INFO:root:Após a realização da operação P8, a subchave K1 resultante é 10100100
INFO:root:Após a realização da operação S2, a sequência resultante é 0010000011
INFO:root:Após a realização da operação P8, a subchave K2 resultante é 01000011
INFO:root:Encriptando a mensagem 11010111
INFO:root:Após a realização da operação IP, a sequência resultante é 11011101
INFO:root:	Rodada Feistel de 11011101 com subchave 10100100
INFO:root:	Após a realização da operação EP, a sequência resultante é 11101011
INFO:root:	Após a realização da operação XOR, a sequência resultante é 01001111
INFO:root:	Após a realização da operação S1, a sequência resultante é 11
INFO:root:	Após a realização da operação S2, a sequência resultante é 11
INFO:root:	Após a realização da operação P4, a sequência 

In [ ]:
class CBC(ModoOperacao):
    def set_vetor_inicial(self, vetor_inicial):
        self.vetor_inicial = vetor_inicial

    def set_tamanho_bloco(self, tamanho_bloco):
        self.tamanho_bloco = tamanho_bloco

    def gerar_vetor_inicial(self):
        return numerico_para_binario(random.getrandbits(self.tamanho_bloco)).zfill(self.tamanho_bloco)

    def encriptar(self, M: str) -> str:
        C = ''
        C_anterior = self.vetor_inicial
        for i in range(0, len(M), self.tamanho_bloco):
            C_anterior = self.cifra.Enc(xor(M[i:i+self.tamanho_bloco], C_anterior))
            C += C_anterior
        return C

    def decriptar(self, C: str) -> str:
        M = ''
        C_anterior = self.vetor_inicial
        for i in range(0, len(C), self.tamanho_bloco):
            M_atual = self.cifra.Dec(C[i:i+self.tamanho_bloco])
            M += xor(C_anterior, M_atual)
            C_anterior = C[i:i+self.tamanho_bloco]
        return M

In [ ]:
sdes = SDES()
sdes.set_key('1010000010')
cbc = CBC(sdes)
cbc.set_tamanho_bloco(8)
cbc.set_vetor_inicial('01010101')
M = '11010111011011001011101011110000'
C = cbc.encriptar(M)
M_ = cbc.decriptar(C)
assert M == M_

INFO:root:Gerando subchaves a partir da chave 1010000010
INFO:root:Após a realização da operação P10, a sequência resultante é 1000001100
INFO:root:Após a realização da operação S1, a sequência resultante é 0000111000
INFO:root:Após a realização da operação P8, a subchave K1 resultante é 10100100
INFO:root:Após a realização da operação S2, a sequência resultante é 0010000011
INFO:root:Após a realização da operação P8, a subchave K2 resultante é 01000011
INFO:root:Encriptando a mensagem 10000010
INFO:root:Após a realização da operação IP, a sequência resultante é 00010001
INFO:root:	Rodada Feistel de 00010001 com subchave 10100100
INFO:root:	Após a realização da operação EP, a sequência resultante é 10000010
INFO:root:	Após a realização da operação XOR, a sequência resultante é 00100110
INFO:root:	Após a realização da operação S1, a sequência resultante é 00
INFO:root:	Após a realização da operação S2, a sequência resultante é 11
INFO:root:	Após a realização da operação P4, a sequência 

# Referências